In [1]:
!python -V

Python 3.13.2


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error


In [28]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")

2025/04/09 12:12:02 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1744180922952, experiment_id='1', last_update_time=1744180922952, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [21]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
train_data_uri = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet"
val_data_uri = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"
df_train = read_dataframe(train_data_uri)
df_val = read_dataframe(val_data_uri)

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [22]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715207732648

In [24]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [29]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", train_data_uri)
    mlflow.log_param("valid-data-path", val_data_uri)

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

🏃 View run bold-perch-363 at: http://127.0.0.1:5000/#/experiments/1/runs/bde0c8ab27ff454786678b326ebe0b20
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [30]:
import xgboost as xgb

In [31]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [32]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [33]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [34]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[0]	validation-rmse:11.35071                          
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [12:13:16] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:10.61105                          
[2]	validation-rmse:9.97955                           
[3]	validation-rmse:9.44309                           
[4]	validation-rmse:8.98967                           
[5]	validation-rmse:8.60807                           
[6]	validation-rmse:8.28820                           
[7]	validation-rmse:8.02142                           
[8]	validation-rmse:7.79940                           
[9]	validation-rmse:7.61403                           
[10]	validation-rmse:7.46027                          
[11]	validation-rmse:7.33318                          
[12]	validation-rmse:7.22754                          
[13]	validation-rmse:7.13989                          
[14]	validation-rmse:7.06700                          
[15]	validation-rmse:7.00695                          
[16]	validation-rmse:6.95612                          
[17]	validation-rmse:6.91230                          
[18]	validation-rmse:6.87513                          
[19]	valid

/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [12:13:57] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.29663                                                    
[1]	validation-rmse:7.86015                                                    
[2]	validation-rmse:7.19026                                                    
[3]	validation-rmse:6.86965                                                    
[4]	validation-rmse:6.72837                                                    
[5]	validation-rmse:6.64562                                                    
[6]	validation-rmse:6.60214                                                    
[7]	validation-rmse:6.57786                                                    
[8]	validation-rmse:6.55853                                                    
[9]	validation-rmse:6.54487                                                    
[10]	validation-rmse:6.53737                                                   
[11]	validation-rmse:6.53173                                                   
[12]	validation-rmse:6.53026            

...

[8]	validation-rmse:8.97652                                                    
[9]	validation-rmse:8.75871                                                    
[10]	validation-rmse:8.56168                                                   
[11]	validation-rmse:8.38447                                                   
[12]	validation-rmse:8.22403                                                   
[13]	validation-rmse:8.07922                                                   
[14]	validation-rmse:7.94932                                                   
[15]	validation-rmse:7.83224                                                   
[16]	validation-rmse:7.72660                                                   
[17]	validation-rmse:7.63173                                                   
[18]	validation-rmse:7.54701                                                   
[19]	validation-rmse:7.46959                                                   
[20]	validation-rmse:7.40094            

/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [12:16:24] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.44375                                                    
[1]	validation-rmse:7.15410                                                    
[2]	validation-rmse:6.74097                                                    
[3]	validation-rmse:6.59639                                                    
[4]	validation-rmse:6.53788                                                    
[5]	validation-rmse:6.50969                                                    
[6]	validation-rmse:6.49287                                                    
[7]	validation-rmse:6.48216                                                    
[8]	validation-rmse:6.47446                                                    
[9]	validation-rmse:6.46873                                                    
[10]	validation-rmse:6.46394                                                   
[11]	validation-rmse:6.45528                                                   
[12]	validation-rmse:6.45213            

/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [12:16:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.96228                                                    
[1]	validation-rmse:6.61689                                                    
[2]	validation-rmse:6.56822                                                    
[3]	validation-rmse:6.55276                                                    
[4]	validation-rmse:6.54275                                                    
[5]	validation-rmse:6.52429                                                    
[6]	validation-rmse:6.51500                                                    
[7]	validation-rmse:6.50916                                                    
[8]	validation-rmse:6.50508                                                    
[9]	validation-rmse:6.49816                                                    
[10]	validation-rmse:6.49184                                                   
[11]	validation-rmse:6.48520                                                   
[12]	validation-rmse:6.48252            

In [36]:
mlflow.xgboost.autolog(disable=True)

In [37]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [12:22:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [12:22:49] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/04/09 12:22:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run adventurous-foal-116 at: http://127.0.0.1:5000/#/experiments/1/runs/00654d27bc124f169f7cb06f56fe07e2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [38]:
import mlflow
logged_model = 'runs:/00654d27bc124f169f7cb06f56fe07e2/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [39]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 00654d27bc124f169f7cb06f56fe07e2

In [40]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [41]:
xgboost_model

In [43]:
y_pred = xgboost_model.predict(valid)
y_pred

array([14.782765 ,  7.184751 , 15.971323 , ..., 13.464008 ,  6.7352147,
        8.184539 ], shape=(61921,), dtype=float32)

In [44]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

🏃 View run vaunted-asp-867 at: http://127.0.0.1:5000/#/experiments/1/runs/088cfce74206467aa57a8faece971683
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run burly-eel-103 at: http://127.0.0.1:5000/#/experiments/1/runs/a900abc342ec4383a013966a547dfd4d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run amusing-kit-963 at: http://127.0.0.1:5000/#/experiments/1/runs/8723e7f5e2e54c63badd11d840de214a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


🏃 View run placid-shark-28 at: http://127.0.0.1:5000/#/experiments/1/runs/9c4aab4451d141ee8251e23f313ea5f0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
